In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import os
from selenium.webdriver.support import expected_conditions as EC
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
import numpy as np

In [2]:
chrome_path = "C:/Users/apple/Desktop/chromedriver_win32/chromedriver.exe"
driver = webdriver.Chrome(chrome_path)

In [3]:
def site_login():
    driver.get("https://portfoliomanager.energystar.gov/pm/login.html")
    driver.find_element_by_id("j_username").send_keys("WASSERMAN")
    driver.find_element_by_id ("j_password").send_keys("Energy2018#")
    driver.find_element_by_id("button-login").click()
site_login()

In [4]:
def wait_load_function(element,page):
    delay = 3 # seconds
    try:        
        myElem = WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.XPATH, element)))
        print(page, "is ready!")
    except TimeoutException:
        print("Loading took too much time!")

In [5]:

def download_excel(n_building):
    buildings = driver.find_elements_by_css_selector("a.ng-binding") # need re-gain element every iteration

    bld = buildings[n_building]
    folder_name = bld.text
    print(folder_name, 'starting!')
    bld.click()
    # do things #
    wait_load_function(element='//*[@id="tab-content-tabs"]/li[3]/a',page = "building homepage") # go to the building homepage

    driver.find_element_by_xpath('//*[@id="tab-content-tabs"]/li[3]/a').click() # go to energy tab
    n_meters = len(driver.find_elements_by_css_selector('td.tr-breakword'))
    print('meter number:',n_meters)
    for j in range(n_meters): # iteration through energy meters (there is 1 water meter)
        time.sleep(1)
        meters = driver.find_elements_by_css_selector('td.tr-breakword')
        meter = meters[j]        
        meter.click()

        wait_load_function(element="""//*[@id="frm-data"]/div[3]/div/div[2]/div[2]/div/ul""", page = "meter page")
        print("meter",j+1,"is ready")

        driver.find_element_by_xpath("""//*[@id="frm-data"]/div[3]/div/div[2]/div[2]/div/ul""").click() # click select field
        driver.find_element_by_xpath("""//*[@id="frm-data"]/div[3]/div/div[2]/div[2]/div/div/ul/li[1]""").click() # show all year

        time.sleep(0.7)

        driver.find_element_by_xpath("""//*[@id="frm-data"]/div[3]/div/div[2]/div[6]/div[2]/span[2]/a""").click() # download
        driver.find_element_by_xpath("""//*[@id="meterDownloadModal"]/div[4]/button/span""").click() # download excel


        driver.back()
        # need to exclude water

    # do things #



    # go back to home

    driver.back()
    driver.back()

    


In [12]:
n_building = 9 # which building 
# 10 done
download_excel(n_building=n_building)

225 West End Avenue starting!
building homepage is ready!
meter number: 9


NameError: name 'TimeoutException' is not defined

In [8]:
folder_name = driver.find_elements_by_css_selector("a.ng-binding")[n_building].text
download_path = "C:/Users/apple/Downloads/"
new_path = "C:/Users/apple/Desktop/EIS/BM/" + folder_name + '/'
try:
    os.mkdir(new_path) # create a folder with building name
    print(folder_name,"created")
except FileExistsError:
    print(folder_name,'already exists')
    

225 West End Avenue already exists


In [86]:
# move files
for files in os.listdir(download_path):
    try:
        os.rename(download_path + files, new_path + files)
    except FileExistsError:
        print('file already exists')
# finish removing

In [13]:
import pandas as pd
os.chdir(new_path)

In [14]:
meters = {}
for i,file in enumerate(os.listdir(new_path)):
    df = pd.read_excel(file, header=None)
    head = df.loc[0,0]
    meters[head] = df
    

In [15]:
len(meters)

8

In [16]:
for key,value in meters.items():
    print(key)

Meter: END                 AVEN  W        225/E/117 (ID 46383013)
Meter: END                 AVEN  W        225/E/2 (ID 46383015)
Meter: END                 AVEN  W        225/E/9 (ID 46383017)
Meter: END                 AVEN  W        225/E/1 (ID 46383010)
Meter: END                 AVEN  W        225/G/101 (ID 46383021)
Meter: END                 AVEN  W        225/G/2 (ID 46383024)
Meter: END                 AVEN  W        225/G/931 (ID 46383025)
Meter: END                 AVEN  W        225/G/1 (ID 46383019)


In [17]:
# organize df
for key in meters.keys():
    meters[key] = meters[key].iloc[5:,1:4]
    meters[key].columns = meters[key].iloc[0].values
    meters[key] = meters[key].drop(5)
    meters[key] = meters[key].reset_index(drop=True)

In [46]:
# apt: E1
# pub: E2, E21
# gas: G2
# oil
apt = {}
pub = {}
gas = {}
oil = {}

In [47]:
for key,value in meters.items():
    if 'E/1 (' in key:
        apt[key] = value 
        
    elif 'E/2' in key or 'E/21' in key or 'E/9' in key or 'E/117' in key:
        pub[key] = value
        
    elif 'G/1' in key or 'G/2' in key or 'G/101' in key or 'G/931' in key:
        gas[key] = value

    elif 'Oil' in key:
        oil[key] = value
        

In [50]:
print('apt')
for key in apt.keys():
    print(key)
print('\npub')
for key in pub.keys():
    print(key)
print('\ngas')
for key in gas.keys():
    print(key)

apt
Meter: END                 AVEN  W        225/E/1 (ID 46383010)

pub
Meter: END                 AVEN  W        225/E/117 (ID 46383013)
Meter: END                 AVEN  W        225/E/2 (ID 46383015)

gas
Meter: END                 AVEN  W        225/G/101 (ID 46383021)
Meter: END                 AVEN  W        225/G/2 (ID 46383024)
Meter: END                 AVEN  W        225/G/931 (ID 46383025)
Meter: END                 AVEN  W        225/G/1 (ID 46383019)


In [51]:
# cutting data to exact 2 years
for key in apt.keys():
    if len(apt[key]) > 24:
        if apt[key].iloc[0,0].month == 12 and apt[key].iloc[0,0].day < 15:
            apt[key] = apt[key].drop(0,axis=0)
        if apt[key].iloc[-1,0].month == 12 and apt[key].iloc[0,0].day > 15:
            apt[key] = apt[key].drop(len(apt[key])-1,axis=0)
        assert apt[key].shape[0] == 24
    
for key in pub.keys():
    if len(pub[key]) > 24:
        if pub[key].iloc[0,0].month == 12 and pub[key].iloc[0,0].day < 15:
            pub[key] = pub[key].drop(0,axis=0)
        if pub[key].iloc[-1,0].month == 12 and pub[key].iloc[0,0].day > 15:
            pub[key] = pub[key].drop(len(pub[key])-1,axis=0)
        assert pub[key].shape[0] == 24

for key in gas.keys():
    if len(gas[key]) > 24:
        if gas[key].iloc[0,0].month == 12 and gas[key].iloc[0,0].day < 15:
            gas[key] = gas[key].drop(0,axis=0)
        if gas[key].iloc[-1,0].month == 12 and gas[key].iloc[0,0].day > 15:
            gas[key] = gas[key].drop(len(gas[key])-1,axis=0)
        assert gas[key].shape[0] == 24
    

In [52]:
# apt_sum
# pub_sum
# gas_sum
# oil_sum
apt_sum = apt[list(apt.keys())[0]][['Start Date', 'End Date']]
pub_sum = pub[list(pub.keys())[0]][['Start Date', 'End Date']]
gas_sum = gas[list(gas.keys())[0]][['Start Date', 'End Date']]
# oil_sum = oil[list(oil.keys())[0]][['Start Date', 'End Date']]


In [53]:
for key in apt.keys():
    apt_sum = pd.merge(apt_sum,apt[key],on = ['Start Date','End Date'], how='outer').sort_values('Start Date')   

for key in pub.keys():
    pub_sum = pd.merge(pub_sum,pub[key],on = ['Start Date','End Date'], how='outer').sort_values('Start Date')    

for key in gas.keys():
    gas_sum = pd.merge(gas_sum,gas[key],on = ['Start Date','End Date'], how='outer').sort_values('Start Date')      

In [54]:
pub_sum['total'] = pub_sum.iloc[:,2:].sum(axis=1)
pub_sum['rate'] = [0.19] * 12 + [0.158962092,
0.182056383,
0.187205935,
0.174078024,
0.23673268,
0.224479825,
0.19231965,
0.200345249,
0.199931593,
0.18597977,
0.153516765,
0.206514726]
pub_sum['cost'] = pub_sum['total'] * pub_sum['rate']
pub_sum[12:]['cost'].sum()

6954.726633224

In [55]:
apt_sum['total'] = apt_sum.iloc[:,2:].sum(axis=1)
apt_sum['rate'] = [0.3] * 12 + [0.31] * 12
apt_sum['cost'] = apt_sum['total'] * apt_sum['rate']
apt_sum[12:]['cost'].sum()

46350.89

In [56]:
gas_sum['total'] = gas_sum.iloc[:,2:].sum(axis=1)
gas_sum['rate'] = [1.13] * 12 + [1.16] * 12
gas_sum['cost'] = gas_sum['total'] * gas_sum['rate']
gas_sum[12:]['cost'].sum()

ValueError: Length of values does not match length of index

In [62]:
print('pub')
print(pub_sum['total'][0:12].sum())
print(pub_sum['total'][12:24].sum())
print('\n')
print('apt')
print(apt_sum['total'][0:12].sum())
print(apt_sum['total'][12:24].sum())
print('\n')

pub
35670.0
36223.0


apt
157072.0
149519.0


149519.0

In [27]:
apt_sum

,Start Date,End Date,Usage kWh (thousand Watt-hours)_x,Usage kWh (thousand Watt-hours)_y,total,rate,cost
0,2015-12-28 00:00:00,2016-01-27 00:00:00,2688,11044,13732.0,0.30,4119.60
1,2016-01-27 00:00:00,2016-02-26 00:00:00,2688,12195,14883.0,0.30,4464.90
2,2016-02-26 00:00:00,2016-03-28 00:00:00,2827,10737,13564.0,0.30,4069.20
3,2016-03-28 00:00:00,2016-04-26 00:00:00,2662,9897,12559.0,0.30,3767.70
4,2016-04-26 00:00:00,2016-05-25 00:00:00,2661,9278,11939.0,0.30,3581.70
5,2016-05-25 00:00:00,2016-06-24 00:00:00,2750,15090,17840.0,0.30,5352.00
6,2016-06-24 00:00:00,2016-07-26 00:00:00,2954,20783,23737.0,0.30,7121.10
7,2016-07-26 00:00:00,2016-08-24 00:00:00,2677,19890,22567.0,0.30,6770.10
8,2016-08-24 00:00:00,2016-09-23 00:00:00,2738,16572,19310.0,0.30,5793.00
9,2016-09-23 00:00:00,2016-10-25 00:00:00,2889,10641,13530.0,0.30,4059.00


In [33]:
pub_sum

,Start Date,End Date,Usage kWh (thousand Watt-hours)_x,Usage kWh (thousand Watt-hours)_y,total,rate,cost
0,2015-12-28 00:00:00,2016-01-27 00:00:00,126,8442,8568.0,0.190000,1627.920000
1,2016-01-27 00:00:00,2016-02-26 00:00:00,234,9810,10044.0,0.190000,1908.360000
2,2016-02-26 00:00:00,2016-03-28 00:00:00,180,9072,9252.0,0.190000,1757.880000
3,2016-03-28 00:00:00,2016-04-26 00:00:00,144,8640,8784.0,0.190000,1668.960000
4,2016-04-26 00:00:00,2016-05-25 00:00:00,126,8568,8694.0,0.190000,1651.860000
5,2016-05-25 00:00:00,2016-06-24 00:00:00,126,8946,9072.0,0.190000,1723.680000
6,2016-06-24 00:00:00,2016-07-26 00:00:00,274,9882,10156.0,0.190000,1929.640000
7,2016-07-26 00:00:00,2016-08-24 00:00:00,374,8856,9230.0,0.190000,1753.700000
8,2016-08-24 00:00:00,2016-09-23 00:00:00,326,9018,9344.0,0.190000,1775.360000
9,2016-09-23 00:00:00,2016-10-25 00:00:00,192,8856,9048.0,0.190000,1719.120000


In [63]:
gas_sum

,Start Date,End Date,Usage therms_x,Usage therms_y,Usage therms_x,Usage therms_y,total
0,2015-12-28,2016-01-27,4,69,NaN,NaN,73.0
25,2015-12-28,2016-01-27,NaN,NaN,NaN,48,48.0
1,2016-01-27,2016-02-26,6,67,NaN,NaN,73.0
26,2016-01-27,2016-02-26,NaN,NaN,NaN,70,70.0
27,2016-02-26,2016-03-28,NaN,NaN,NaN,60,60.0
2,2016-02-26,2016-03-28,6,66,NaN,NaN,72.0
28,2016-03-28,2016-04-26,NaN,NaN,NaN,55,55.0
3,2016-03-28,2016-04-26,4,70,NaN,NaN,74.0
29,2016-04-26,2016-05-25,NaN,NaN,NaN,48,48.0
4,2016-04-26,2016-05-25,5,63,NaN,NaN,68.0


In [437]:
# delete E/26?